In [1]:
# https://www.kaggle.com/kyakovlev/1st-place-solution-part-1-hands-on-data

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
_base_dir = os.path.join(os.getcwd(),'..','dataset')
_path_test_csv = os.path.join(_base_dir,'test.csv')

dataset = pd.read_csv(os.path.join(_base_dir,'dataset.csv'))
test_data = pd.read_csv(_path_test_csv)
sample_submission = pd.read_csv(os.path.join(_base_dir,'sample_submission.csv'))

In [93]:
group_by_month = dataset.groupby('date_block_num')
group_by_oct = group_by_month.get_group(33).groupby(['item_id','shop_id']).aggregate({'item_cnt_day':np.sum})
group_by_oct_DOW = group_by_month.get_group(33).groupby(['item_id','shop_id','DOW']).aggregate({'item_cnt_day':np.sum})

In [190]:
def get_val_of_oct(item,shop):
    try:return(group_by_oct.loc[item,shop].values[0])
    except:return 0
def get_val_of_oct_DOW(item,shop,DOW):
    try:return(group_by_oct_DOW.loc[item,shop].loc[DOW,].values[0])
    except:return get_val_of_oct(item,shop)

#### checking for data in september 

In [121]:
test_data_sub = test_data.copy()
test_data_sub['item_cnt_month'] = test_data_sub.apply(lambda x:get_val_of_oct(x.item_id,x.shop_id),axis=1)

In [123]:
test_data_sub.drop(columns=['shop_id','item_id'],inplace=True)

In [124]:
test_data_sub.item_cnt_month.clip(0,20,inplace=True)
test_data_sub.to_csv('submission.csv',index=False)

#### previous data benchmark 2

In [191]:
test_data_sub = test_data.copy()
test_data_sub['DOW'] = np.array(range(6,test_data.shape[0]+6))%7
test_data_sub['item_cnt_month'] = test_data_sub.apply(lambda x:get_val_of_oct_DOW(x.item_id,x.shop_id,x.DOW),axis=1)

In [192]:
test_data_sub.head()

,ID,shop_id,item_id,DOW,item_cnt_month
0,0,5,5037,6,0.0
1,1,5,5320,0,0.0
2,2,5,5233,1,1.0
3,3,5,5232,2,0.0
4,4,5,5268,3,0.0


In [194]:
test_data_sub.drop(columns=['shop_id','item_id'],inplace=True)
test_data_sub.drop(columns=['DOW'],inplace=True)
test_data_sub.item_cnt_month.clip(0,20,inplace=True)
test_data_sub.to_csv('submission_2.csv',index=False)

#### using xgboost algorithm

In [200]:
import xgboost as xgb

In [201]:
model = xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1)

In [210]:
dataset.head()
xgb_train = dataset.loc[:,['date_block_num','shop_id','item_id']].values
xgb_test = dataset.loc[:,'item_cnt_day'].values

In [211]:
model.fit(xgb_train, xgb_test, eval_metric='rmse')

[10:17:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=0.5, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, num_round=1000, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1, silent=None, subsample=1, verbosity=1)

In [219]:
xgb_test_x = test_data.loc[:,['shop_id','item_id']]
xgb_test_x['date_block_num'] = 34
xgb_test_y = model.predict(xgb_test_x.values)

In [227]:
xgb_test_sub = test_data.copy()
xgb_test_sub['item_cnt_month'] = xgb_test_y
xgb_test_sub.drop(columns=['shop_id','item_id'],inplace=True)
xgb_test_sub['item_cnt_month'].clip(0,20,inplace=True)

In [228]:
xgb_test_sub.to_csv('submission_xgb_1.csv',index=False)

In [226]:
xgb_test_sub

,ID,item_cnt_month
0,0,0.967638
1,1,0.967638
2,2,0.967638
3,3,0.967638
4,4,0.967638
...,...,...
214195,214195,0.986681
214196,214196,0.986681
214197,214197,0.986681
214198,214198,0.986681


#### xgboost try 2 with more values

In [248]:
dataset.head()

,Unnamed: 0,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue,item_category_val,DOW
0,0,2013-01-02,0,59,22154,999.00,1.0,37,999.00,0.0,2
1,1,2013-01-03,0,25,2552,899.00,1.0,58,899.00,1.0,3
2,2,2013-01-05,0,25,2552,899.00,-1.0,58,-899.00,1.0,5
3,3,2013-01-06,0,25,2554,1709.05,1.0,58,1709.05,1.0,6
4,4,2013-01-15,0,25,2555,1099.00,1.0,56,1099.00,6.0,1


In [302]:
xgb2_train_x = dataset.loc[:,['date_block_num','shop_id','item_id','item_category_val']].values
xgb2_train_y = dataset.loc[:,'item_cnt_day'].values
model.fit(xgb2_train_x, xgb2_train_y, eval_metric='rmse')

[12:16:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=0.5, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, num_round=1000, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1, silent=None, subsample=1, verbosity=1)

In [280]:
xgb2_test_x = test_data.copy()
xgb2_test_x['date_block_num'] = 34
xgb2_test_x = pd.merge(xgb2_test_x,dataset[['date_block_num','shop_id','item_id','item_category_val']],on='item_id',how='left',copy=False)

In [281]:
xgb2_test_x = xgb2_test_x.drop_duplicates(['shop_id_x','item_id'])

In [288]:
xgb2_test_x = xgb2_test_x.drop(columns=['date_block_num_y','shop_id_y'])

In [289]:
xgb2_test_x.rename(columns = {'shop_id_x':'shop_id','date_block_num_x':'date_block_num'},inplace=True)

In [311]:
xgb2_test_y = model.predict(xgb2_test_x.iloc[:,1:].fillna(0).values)

In [312]:
xgb2_test_sub = test_data.copy()
xgb2_test_sub['item_cnt_month'] = xgb2_test_y
xgb2_test_sub.drop(columns=['shop_id','item_id'],inplace=True)
xgb2_test_sub['item_cnt_month'].clip(0,20,inplace=True)

In [314]:
xgb2_test_sub.to_csv('submission_xgb_2.csv',index=False)